In [3]:
import xarray as xr
import s3fs
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import cartopy.crs as ccrs
import cartopy
import matplotlib.path as mpath
from functions import compute_ivt,to_nc
from matplotlib import rc,animation
from matplotlib.animation import FuncAnimation
from IPython import display

In [4]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [5]:
cat = col.search(source_id=['NorESM2-LM'], experiment_id=['ssp245'], table_id=['day'], variable_id=['pr','clt','hus','va','tas'], member_id=['r1i1p1f1'])
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,va,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
1,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,tas,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
2,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,pr,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
3,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,hus,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108
4,ScenarioMIP,NCC,NorESM2-LM,ssp245,r1i1p1f1,day,clt,gn,gs://cmip6/CMIP6/ScenarioMIP/NCC/NorESM2-LM/ss...,NaN,20191108


In [6]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [7]:
dataset_list = list(dset_dict.keys())

In [13]:
start_year = 2035
end_year = 2050
cm245 = dset_dict[dataset_list[0]]
#dset = dset.sel(member_id='r1i1p1f1',time=slice("2000-01-01", "2014-12-31"))
cm245_3550 = cm245.sel(time = slice(str(start_year)+"-01-01", str(end_year)+"-12-31"))

In [15]:
cm245_3550

<xarray.Dataset>
Dimensions:    (member_id: 1, time: 5840, lat: 96, lon: 144, bnds: 2, plev: 8)
Coordinates:
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(96, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(144, 2), meta=np.ndarray>
  * time       (time) object 2035-01-01 12:00:00 ... 2050-12-31 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(5840, 2), meta=np.ndarray>
  * member_id  (member_id) <U8 'r1i1p1f1'
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    clt        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 1205, 96, 144), meta=np.ndarray>
    hus        (member_id, time, plev, lat, lon) float32 dask.array<chunksize=(1, 80, 8, 96, 144), meta=np.ndarray>
    pr         (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 190, 96, 144), meta=np.ndarray>
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 305, 96, 144), meta=np.ndarray>
    va         (member_id, time, plev, lat, lon) float32 dask.array<chunksize=(1, 136, 8, 96, 144), meta=np.ndarray>
Attributes: (12/54)
    realization_index:         1
    intake_esm_varname:        clt\nhus\npr\ntas\nva
    status:                    2020-07-27;created; by gcs.cmip6.ldeo@gmail.com
    experiment_id:             ssp245
    parent_mip_era:            CMIP6
    grid_label:                gn
    ...                        ...
    sub_experiment_id:         none
    parent_activity_id:        CMIP
    parent_experiment_id:      historical
    tracking_id:               hdl:21.14100/ee54a00b-1513-4379-89f0-2315614dc...
    netcdf_tracking_ids:       hdl:21.14100/ab624c9a-12a7-43da-b6bf-b42b9e949...
    intake_esm_dataset_key:    ScenarioMIP.NCC.NorESM2-LM.ssp245.day.gn

In [16]:
# import AOD
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", 
                       secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0",
                       client_kwargs=dict(endpoint_url="https://rgw.met.no"))
#s3.ls('escience2022/Remy/')

['escience2022/Remy/.keep',
 'escience2022/Remy/.wget-20221102193645.sh.status',
 'escience2022/Remy/.wget-20221102193729.sh.status',
 'escience2022/Remy/.wget-20221102193735.sh.status',
 'escience2022/Remy/.wget-20221102193742.sh.status',
 'escience2022/Remy/Era5',
 'escience2022/Remy/dl_era5_hus.py',
 'escience2022/Remy/dl_era5_wind.py',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19500101-19591231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19600101-19691231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19700101-19791231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19800101-19891231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_19900101-19991231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_20000101-20091231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_20100101-20141231.nc',
 'escie

In [17]:
# import AOD ssp245 for 2035-2050
s3path = list([
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20310101-20401231.nc',
 'escience2022/Remy/od550aer_AERday_NorESM2-LM_ssp245_r1i1p1f1_gn_20410101-20501231.nc'
])

sopenlist=[s3.open(ss) for ss in s3path]

aod245_3550 = (xr.open_mfdataset(sopenlist)).sel(time = slice(str(start_year)+"-01-01", str(end_year)+"-12-31"))